# Regression with Keras

In [ ]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

## Generate a dataset

Generate the dataset. We work with one-dimensional feature vectors, with feature values generated uniformly in $[-5,5]$. The target values will be normally distributed (mean $\mu = 0$, standard deviation $\sigma=50$) around a cubic function of the feature values, with the law

$$
y = F(x) = x^3 - \frac{1}{2}\,x^2 + 20\,x
$$

In [ ]:
X = np.random.uniform(-5.0, 5.0, size=(1000,1))
Y = np.random.normal(X**3 - 5.0*X**2 + 20.0*X, 50.0)

In [ ]:
trace = go.Scatter(
    x = X[:,0],
    y = Y[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    )
)

data = [trace]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    )
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

## Build the neural network with Keras

Keras offers an high-level interface to the underlying backend (TensorFlow in this case). A neural network is Keras is defined as a `model` object to which we can add layers to build the graph. Here we work with a feed-forward neural network built with fully connected `Dense` layers.

For general advice on nonlinear regression using Keras, see:
- https://github.com/keras-team/keras/issues/1874
- https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import mae

For the first layer of the graph the `input_shape` keyword argument must be specified. This is a tuple of integers and does __not__ include the sample axis. Keras expects the input to have the form (n_samples, \[features\]), and `input_shape` must correspond to the shape of the features tensor.

__Example:__ if we have 50 samples, each of which is an array (tensor of rank 1 in TensorFlow terms) with 10 components, we must specify `input_shape=(10,)`.

The dimension of the last layer decides the shape of the output. Here we start with 1-dimensional inputs and we end up with 1-dimensional outputs, as we have a function $F: \mathbb{R}\to\mathbb{R}$. The hidden (intermediate) layers, on the other hand, are wider.

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.add(Dense(50, activation='tanh'))
model.add(Dense(5))
model.add(Dense(1))

A common issue when doing regression with neural network is the __exploding gradients problem__, where the loss function diverges during the training phase. In this case, it was sufficient to switch from stochastic gradient descent to an Adam optimizer to solve that. The Adam optimizer is a generalization of stochastic gradient descent that exploits an adaptive learning rate.

For information about the exploding gradients problem, see:
- https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
- http://neuralnetworksanddeeplearning.com/chap5.html

For information about the Adam optimizer, see: https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

To initialize the neural network we just call a model's `compile()` method. This include the initialization of all the weights of the layers.

In [ ]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mse']
)

A model's `summary()` method gives an overview of the structure of the graph. In particular, from this we can see the number of parameters the neural network possesses.

In [ ]:
model.summary()

## Fit the neural network to the data

Keras provides an interface for training and predicting completely analogous to that of a SKLearn model. Among the options, we can specify the `epochs` parameters.

__Epochs:__ number of passages over the whole dataset the neural network will do in the training phase.

Since the dataset is small we are not dividing it in batches, meaning that each gradient descent (or Adam) step uses the exact gradient descent. This also implies that there are no stochastic features in the path followed in parameters space.

In [ ]:
model.fit(X, Y, epochs=500)

## Generate a new dataset and get predictions

To check whether the neural network did a good job, we can make prediction over a (1-dimensional) grid of values and compare it with the true value of the function to fit on the same grid.

In [ ]:
X_pred = np.linspace(-10.0, 10.0, 1000).reshape((1000,1))
Y_pred = model.predict(X_pred)

In [ ]:
def f_to_fit(x):
    return x**3 - 5.0*x**2 + 20.0*x

In [ ]:
trace_data = go.Scatter(
    x = X[:,0],
    y = Y[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'data'
)

trace_pred = go.Scatter(
    x = X_pred[:,0],
    y = Y_pred[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'predictions'
)

trace_true = go.Scatter(
    x = X_pred[:,0],
    y = f_to_fit(X_pred)[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'true values'
)

data = [trace_data, trace_pred, trace_true]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    ),
    hovermode = 'closest'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Comments: overfitting?

A question is in order on whether by looking at the plot above we should think we are overfitting the data. Notice that no cross validation (train-test split) has been done: we are just comparing the predictions from the neural network with the true, deterministic form of the function we wanted to fit. Ultimately, the question we would like to answer is: __given the data, can the neural network learn the true form of the function?__

In the region where datapoints are present, the fit to the data is good and there's good correspondence with the deterministic function as well: to see if there's overfitting here we should do cross validation on the dataset.

In the region where no datapoint is present, the predictions do not match the deterministic function. In my view, this is somewhat expected: approximating functions with functions of a different form usually requires series (power series, trigonometric series etc.). The prediction of the neural networks, as a function of the input, is just not a cubic, therefore shouldn't be expected to match it in an arbitrary large domain.

## More experiments

We can tweak the topology of the graph and the number of epochs to see how the results vary. The best generalization seems to be the one using wider layers with `tanh` activation function followed by narrower linear layers.

In [ ]:
model_2 = Sequential()
model_2.add(Dense(1, input_shape=(1,)))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(5))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(5))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(5))
model_2.add(Dense(1))

model_2.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mse']
)

model_2.fit(X, Y, epochs=500)

In [ ]:
Y_pred = model_2.predict(X_pred)

trace_data = go.Scatter(
    x = X[:,0],
    y = Y[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'data'
)

trace_pred = go.Scatter(
    x = X_pred[:,0],
    y = Y_pred[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'predictions'
)

trace_true = go.Scatter(
    x = X_pred[:,0],
    y = f_to_fit(X_pred)[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'true values'
)

data = [trace_data, trace_pred, trace_true]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    ),
    hovermode = 'closest'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

## Regression on no-noise data

Let's see what happens if, instead of generating data with target values distributed (normally) around a deterministic function of the feature values, we train the neural network with data exactly following that function.

In [ ]:
X_det = np.linspace(-5.0, 5.0, 1000).reshape((1000,1))
Y_det = f_to_fit(X_det)

In [ ]:
trace = go.Scatter(
    x = X_det[:,0],
    y = Y_det[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    )
)

data = [trace]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    )
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
model_det = Sequential()
model_det.add(Dense(1, input_shape=(1,)))
model_det.add(Dense(10, activation='tanh'))
model_det.add(Dense(5))
model_det.add(Dense(10, activation='tanh'))
model_det.add(Dense(5))
model_det.add(Dense(10, activation='tanh'))
model_det.add(Dense(5))
model_det.add(Dense(1))

In [ ]:
model_det.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mse']
)

model_det.fit(X_det, Y_det, epochs=500)

In [ ]:
X_pred = np.linspace(-10.0, 10.0, 1000).reshape((1000,1))
Y_pred = model_det.predict(X_pred)

trace_data = go.Scatter(
    x = X_det[:,0],
    y = Y_det[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'data'
)

trace_pred = go.Scatter(
    x = X_pred[:,0],
    y = Y_pred[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'predictions'
)

trace_true = go.Scatter(
    x = X_pred[:,0],
    y = f_to_fit(X_pred)[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'true values'
)

data = [trace_data, trace_pred, trace_true]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    ),
    hovermode = 'closest'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Conclusion

Even with data completely devoid of noise, the neural network provides predictions that do not match the true function outside of the region where the datapoint are present. This again signifies that the model does a good job at fitting what it sees but hasn't learned the true underlying rule: you need a cubic to fully understand a cubic!